# (Sounds of Frustration Intensify)

Well, not right away, but Harry says to expect more headaches as the examples get less trivial from now on.  One common issue is that we may see different outputs than he documents in the book.  Some degree of divergence is common when the examples get more complex.

## Django Project Structure $ \ni $ "apps"

(I hope I used that right; I meant to say something like "entails" or "includes"; in [Set notation that may be that a set "owns" the latter term](https://www.geeksforgeeks.org/set-notations-in-latex/).)

The point is that Django expects us to organize semi-autonomous code into elements called "apps", and you can initialize a new app with

`python manage.py startapp lists`

In [14]:
# trying it live; comment out after invoking:
# !python manage.py startapp lists

CommandError: 'lists' conflicts with the name of an existing Python module and cannot be used as an app name. Please try another name.


Which constructs a new file structure and dir:

~~~bash
.
├── 00_prereqs.ipynb
├── 01_django_setup_functional_test.ipynb
├── 02_unittest_module.ipynb
├── 03_home_page.ipynb
├── README.md
├── db.sqlite3
├── functional_tests.py
├── geckodriver.log
├── lists
│   ├── __init__.py
│   ├── admin.py
│   ├── apps.py
│   ├── migrations
│   │   └── __init__.py
│   ├── models.py
│   ├── tests.py
│   └── views.py
├── manage.py
├── superlists
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── __init__.cpython-36.pyc
│   │   ├── settings.cpython-36.pyc
│   │   ├── urls.cpython-36.pyc
│   │   └── wsgi.cpython-36.pyc
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
├── test_dev_book.yml
└── workspace.code-workspace
~~~

Note the major handles and their implied logic: models, views, and tests.

# Unit Tests vs Functional Tests

Then there's a procedural note here about how to think about the two types of tests mentioned so far.  Basically (as mentioned in the previous chapter), Functional tests are meant to recapitulate the gist of the user experience, testing your code "from the outside" and seeing what it gets when a big chunk of it is run.  Unit tests are written from the point of view of the developer, and generally (as I interpret it at this point) test smaller chunks of your code, "from the inside".

He further specifies the general workflow you use in Test-Driven Development:

1. You write a functional test meant to capture how you anticipate the user to interact with your app
2. Once the functional test fails as expected, you think about what code you'd need to get it to pass.  But before you actually write that, you write unit tests to capture how you expect that code to work, and how those lines themselves could fail.
3. When you have a failing unit test for what you expect to get you past (part of) the functional test, you write the smallest unit of *application code* that would do something useful.
4. Then you iterate over steps 2 and 3 as needed until your functional test can pass.

<div class="alert alert-info">I'll just quote verbatim here the last text of this section which takes yet another stab at contextualizing the distinction between these two kinds of tests: "<i>Functional tests should help you build an application with the right functionality, and guarantee you never accidentally break it.  Unit tests should help you write code that's clean and bug-free.</i>"</div>

# Unit Testing in Django

We need to modify that `lists/tests.py` file.

In [3]:
# %load lists/tests.py
from django.test import TestCase

# Create your tests here.


Ok.  He notes how they prompt us by importing a class they expect to be useful, the `django.test.TestCase` class, which extends `unittest.TestCase`, adding functionality that may be relevant for creating web apps, specifically.

Since this was added automatically by Django, rather than something we configured ourselves from the outset, Harry suggests adding a kind of crazy/silly test to it to ensure that it's being invoked and behaving as expected when the whole project is tested:

In [7]:
%%writefile lists/tests.py

from django.test import TestCase

class SmokeTest(TestCase):

    def test_bad_math(self):
        self.assertEqual(1 + 1, 3)

Overwriting lists/tests.py


Then you invoke it (turns out this invocation *doesn't* require you fire up the server first in another window with `runserver`) by calling `python manage.py` in the main project, and specifying one additional keyword: `test`:

In [10]:
%run manage.py test

Creating test database for alias 'default'...


F

System check identified no issues (0 silenced).
Destroying test database for alias 'default'...



FAIL: test_bad_math (lists.tests.SmokeTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 7, in test_bad_math
    self.assertEqual(1 + 1, 3)
AssertionError: 2 != 3

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (failures=1)


SystemExit: 1

## Django's MVC, URLs, and View Functions

Ok, then we cover that DJango has a *Model-View-Controller (MVC)* schema.  He says there are more details about how that's implemented in [their docs](https://docs.djangoproject.com/en/1.11/faq/general/).  The major points are that

1. The server receives an HTTP request for a particular URL
2. Django applies some logic to determine which *view function* should deal with that request, which is known as *resolving the URL*
3. Then the view function returns the chosen HTTP response

So we write a test to see how the program handles the root ("`/`") of our page, and put that in the `lists/tests.py` file (and don't mind that it overwrites that trivial/silly test previously committed to that file):

In [15]:
%%writefile lists/tests.py

from django.urls import resolve
from django.test import TestCase
from lists.views import home_page

class HomePageTest(TestCase):

    def test_root_url_resolves_to_home_page_view(self):
        found = resolve('/')
        self.assertEqual(found.func, home_page)

Overwriting lists/tests.py


Then he breaks down the code, saying that the `django.urls.resolve` function handles that second numbered point above: maps the URL to the *view function* that should be returned.  In this case, that function is `home_page`.  So then you're just checking that that is indeed what gets returned as the `func.` attribute of what gets returned by that `resolve` function when called (which we've assigned to the arbitrary variable `found`, in this case).

The function that `resolve` returns in turn has the responsibility to yield the HTML content appropriate to the URL.  So, the third numbered item above.

In [19]:
# %run lists/tests.py
# %run manage.py test
!python manage.py test

Creating test database for alias 'default'...

E
ERROR: lists.tests (unittest.loader._FailedTest)
----------------------------------------------------------------------
ImportError: Failed to import test module: lists.tests
Traceback (most recent call last):
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\unittest\loader.py", line 428, in _find_test_path
    module = self._get_module_from_name(name)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\unittest\loader.py", line 369, in _get_module_from_name
    __import__(name)
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 4, in <module>
    from lists.views import home_page
ImportError: cannot import name 'home_page'


----------------------------------------------------------------------
Ran 1 test in 0.000s

FAILED (errors=1)



System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


Which is to be expected, as there's no `home_page` file that we've written anywhere in this project yet.  This counts as an expected failure, so good practice, still.

---

As one procedural note about the complications of executing this on Windows: the expected result according to Percival is the `ImportError`, and I get that if I call `lists/tests.py` directly via the IPython `%run` magic, but if I instead invoke `manage.py` with that same magic, I instead see the aforementioned `AssertionError: 2 != 3`.  And the book specifically says to call it with 

`python manage.py test`

from a terminal.  And if you instead use the shell magic to call it in that fashion from within the notebook, I *do* get the `ImportError`.  (Perhaps more trivially, I can confirm that if I invoke from within a terminal, it returns the expected `ImportError`, too; so it's only the `%run` magic executed on `manage.py` that confuses things.)  So behavior is a little inconsistent/unpredictable if you try to get clever about orchestrating everything from within a Notebook environment.  So watch out for that, going forward.

---

Moreover, Harry says that we have now written one failing functional test, and one failing unit test.  Now, I've resumed these notes after a couple weeks of absence, so I had to try to re-familiarize myself with which file constituted which kind of test.  As the contents of `lists/tests.py`'s `HomePageTest` inherits from `django.test.TestCase`, which in turn extends `unittest.TestCase`, I figured that should count as a Unit Test, but then even the sole class so far contained in the `functional_tests.py` file ("`NewVistorTest`") also inherits from `unittest.TestCase`, obviously the code base that a given chunk of test code extends is not a fool-proof indicator as to which it is (which maybe should be expected, as I don't think there's any `functionaltest` or similar module in Python).  Instead, I guess it's really all about the overall philosophy as covered above: if the logic within the file sounds like it's emulating how an end user may invoke your app, then it's a **functional** test; if it looks more wonk-ish or opaque from the outside, it's likely a **unit** test.

Anyways, the point is that he's saying that you want to make sure you have an example of an expected failure with both a unit test *and* a functional test before you start writing code about what your application is actually supposed to *do*.  So we're ready to start that, now.

But Harry says he's being a little facetiously or extremely plodding with the pacing here: we don't want to write the whole HTML code for the home page, nor even the entirety of the `home_page` function that's meant to return it.  Instead, we observe painfully incremental change to fix the existing unit test failure (the resulting code goes into the `lists/views.py` file, to accord with the overall architecture that Django expects):

In [21]:
%%writefile lists/views.py

from django.shortcuts import render

# Create your views here.
home_page = None

Overwriting lists/views.py


In [22]:
!python manage.py test

Creating test database for alias 'default'...

E
ERROR: test_root_url_resolves_to_home_page_view (lists.tests.HomePageTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 9, in test_root_url_resolves_to_home_page_view
    found = resolve('/')
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\urls\base.py", line 27, in resolve
    return get_resolver(urlconf).resolve(path)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\urls\resolvers.py", line 394, in resolve
    raise Resolver404({'tried': tried, 'path': new_path})
django.urls.exceptions.Resolver404: {'tried': [[<RegexURLResolver <RegexURLPattern list> (admin:admin) ^admin/>]], 'path': ''}

----------------------------------------------------------------------
Ran 1 test in 0.006s

FAILED (errors=1)



System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


And that appears to match the anticipated result in the text.

<div class="alert alert-info">There's a box here about <b>Reading Tracebacks</b>; he says to start at the bottom with the reported error itself ("<code>django.urls.exceptionsResolver404</code>").  In our case that isn't immediately informative, so you check next for one of the files that <i>you wrote</i> in the traceback, from the top down; specifically, which of your <b>tests</b> failed.  In this case, we expect that the most recent one that we wrote should be at fault: the <code>test_root_url_resolves_to_home_page_view</code> in the <code>lists/tests.py</code> file, which it is.  Then, you look for more context within that test: the call to <code>resolve('/')</code>.  But then the trail kind of goes cold, unless you know enough about how Django works to interpret the downstream stuff.  But it's basically saying in this case that Django can't find the content meant to map to the <code>'/'</code> URL.</div>

## `urls.py`

Harry says that the solution is to write a `superlists/urls.py` file to provide the mappings of URLs to HTML files.  This should've been written by the original 

`python manage.py startapp lists`

call to kick off this notebook; let's check that file's existence and content:

In [24]:
# %load superlists/urls.py
"""superlists URL Configuration

The `urlpatterns` list routes URLs to views. For more information please see:
    https://docs.djangoproject.com/en/1.11/topics/http/urls/
Examples:
Function views
    1. Add an import:  from my_app import views
    2. Add a URL to urlpatterns:  url(r'^$', views.home, name='home')
Class-based views
    1. Add an import:  from other_app.views import Home
    2. Add a URL to urlpatterns:  url(r'^$', Home.as_view(), name='home')
Including another URLconf
    1. Import the include() function: from django.conf.urls import url, include
    2. Add a URL to urlpatterns:  url(r'^blog/', include('blog.urls'))
"""
from django.conf.urls import url
from django.contrib import admin

urlpatterns = [
    url(r'^admin/', admin.site.urls),
]


Then he mentions the general structure of what that `django.conf.urls.url` function expects:

1. A Regex string defining to which kinds of URLs it should apply
2. Where it should send the request: "*either to a view function you've imported, or maybe to another `urls.py` file somewhere else*"

Harry suggests following a couple of the recommendations from the default-formattted file's doc strings.  Specifically, try plopping in a regex for an empty line ("`^$`") and replacing that reference to the `django.contrib.admin.site.urls` func, which he says we won't use.  Instead, substitute the `home_page` func from our `views` module:

In [27]:
%%writefile superlists/urls.py
from django.conf.urls import url
from lists import views

urlpatterns = [
    url(r'^$', views.home_page, name='home'),
]

Overwriting superlists/urls.py


Then, re-try the most recent test:

In [28]:
!python manage.py test

Creating test database for alias 'default'...


Traceback (most recent call last):
  File "manage.py", line 22, in <module>
    execute_from_command_line(sys.argv)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\core\management\__init__.py", line 364, in execute_from_command_line
    utility.execute()
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\core\management\__init__.py", line 356, in execute
    self.fetch_command(subcommand).run_from_argv(self.argv)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\core\management\commands\test.py", line 29, in run_from_argv
    super(Command, self).run_from_argv(argv)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\core\management\base.py", line 283, in run_from_argv
    self.execute(*args, **cmd_options)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\core\management\base.py", line 330, in execute
    output = self.handle(*args, **opt

Wow, that's a whole crap-ton of calls in the traceback.  But anyways, the final line matches what's anticipated from the text.  Harry says it counts as progress, insofar as we're no longer seeing a `404`-type error about your app not finding where to look.

He says that the nature of the error above (that the thing ultimately returned in response to the request is not callable) indicates that the intended connection between the home URL ("`/`") and the `home_page = None` line in `lists/views.py` *is* being made.  So all of that was another very small, cautious step to ensure that your test is working as anticipated, and now justifies replacing `None` in that line with something more interesting.

So the next change is to replace that line in `lists/views.py` with something callable (first, a cell to refresh our knowledge of the preexisting contents of that file):

In [29]:
# %load lists/views.py

from django.shortcuts import render

# Create your views here.
home_page = None


In [30]:
%%writefile lists/views.py

from django.shortcuts import render

# Create your views here.
def home_page():
    pass


Overwriting lists/views.py


And then repeat the test:

In [31]:
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Which constitutes the first passing unit test in the project/app/book.  He suggests it might be a good time to commit to your repo.

Ok, then we edit the `lists/tests.py` file, and add a second test:

In [32]:
# %load lists/tests.py

from django.urls import resolve
from django.test import TestCase
from lists.views import home_page

class HomePageTest(TestCase):

    def test_root_url_resolves_to_home_page_view(self):
        found = resolve('/')
        self.assertEqual(found.func, home_page)


In [33]:
%%writefile lists/tests.py
from django.urls import resolve
from django.test import TestCase
from django.http import HttpRequest

from lists.views import home_page

class HomePageTest(TestCase):

    def test_root_url_resolves_to_home_page_view(self):
        found = resolve('/')
        self.assertEqual(found.func, home_page)

    def test_home_page_returns_correct_html(self):
        request = HttpRequest()
        response = home_page(request)
        html = response.content.decode('utf8')
        self.assertTrue(html.startswith('<html>'))
        self.assertIn('<title>To-Do lists</title>', html)
        self.assertTrue(html.endswith('</html>'))

Overwriting lists/tests.py


Then, we cover what the above is supposed to do:

1. (line 15) The `django.http.HttpRequest` object stores what Django will receive to process when a user issues a request for a URL within your site
2. (line 16) You pass the above `HttpRequest` object to your `home_page` view/function
3. (line 17) You grab the HTML content of the page returned by the above view, by decoding from bytes
4. (lines 18 & 20) You check that the whole decoded page is enclosed within an `<html></html>` tag, to constitute legal HTML
5. (line 19) You check that the title is as expected

At this point, though, the `home_page` view/function written to `lists/views` actually takes no args, so this is an expected failure case:

In [34]:
!python manage.py test

E.
ERROR: test_home_page_returns_correct_html (lists.tests.HomePageTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 15, in test_home_page_returns_correct_html

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...



    response = home_page(request)
TypeError: home_page() takes 0 positional arguments but 1 was given

----------------------------------------------------------------------
Ran 2 tests in 0.001s

FAILED (errors=1)


## The Unit-Test/Code cycle

At this point, Harry says that you can settle into a rhythm: you write a new unit test to check for the successful functioning of the next significant line/chunk of code that you intend to write in your application, check that it fails, then write the application, and check that it succeeds.

What's left unsaid (as I interpret it) is that it's still a matter of habit and art to know *what* to test, exactly.  For instance, I can see why it might be important to check that your returned home page in the above test is enclosed within the `html` tags, but why bother to check that the title is as expected, as opposed to other stuff?  I'm not much of a web developer (well, I'm not *any* of a web developer), so which characteristics are important to check and maintain don't really jump out to me.  And so (again, I interpret) it goes for other applications and code bases: you'd have to be decent at building various types of stuff to know what was important vs trivial to ensure was working in the code.  But so much for my aside...

He does, however, offer that when you start out, your tests will need to be more painfully, laboriously minimal and thorough, with the insinuation that once you get a lot better at a subject (like web development), you'll build up a better working knowledge about what is likely to fail in practice, and maybe focus your written tests on those pain points.  Of course, that's kind of me reading between the lines; he can't really encourage deviation from the *best practice* of writing tests for every little damn thing, because you can't always write off a given failure scenario as irrelevant.

But anyways, at this point we review the above steps, in order to try to build up a habit of knowing what constitutes a good minimal test in this case, based on the application code you expect to write:

* Desired functionality:
    * Your `lists/views.py` needs to include a function to return your home page (but not necessarily actually pass anything *real*, at this point):
        ~~~python
        def home_page(request):
            pass
        ~~~
* The corresponding test to write:
    * Check the content of what gets passed, by decoding to standard HTML:
        ~~~python
        html = response.content.decode('utf8')
        ~~~
* Anticipated outcome:
    * You told your function to just `pass` (minimal incremental functionality is added to application code), so it should raise an error saying `NoneType` lacks the attribute:
        ~~~bash
        AttributeError: 'NoneType' object has no attribute 'content'
        ~~~

* Desired functionality:
    * Once you've ensured the first test fails, augment your `home_page` view to instead at least return an empty `django.http.HttpResponse` object:
        ~~~python
        def home_page(request):
            return HttpResponse
        ~~~
* Corresponding test:
    * Check that it's formatted as HTML:
        ~~~python
        self.assertTrue(html.startswith('<html>'))
        ~~~
* Anticipated outcome:
    * `HttpResponse` objects don't have that tag unless you pass them some kind of content to structure as a valid HTML page, so:
        ~~~python
        AssertionError: False is not True
        ~~~

* Desired functionality:
    * When *that* test fails, move on to giving the `HttpResponse` object some properly-formatted HTML text to sink its teeth into:
        ~~~python
        def home_page(request):
            content = '<html><title>To-Do lists</title></html>'
            return HttpResponse(content)

So now, instead of mapping out hypotheticals in markdown, actually augment the specified files.  The `tests.py` file is up to date, so just update `views.py`:

In [38]:
%%writefile lists/views.py

from django.shortcuts import render
from django.http import HttpResponse

def home_page(request):
    content = '<html><title>To-Do lists</title></html>'
    return HttpResponse(content)


Overwriting lists/views.py


In [39]:
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


So that's our unit tests up-to-date and passing; now he says to check the functional tests:

<div class="alert alert-warning"><p>He notes that you have to "spin up the dev server again"; I've gone long enough between edits to this notebook to forget how to do that.  Turns out it's</p>
<code>python manage.py runserver</code>
<br>
<p>in a separate terminal.  When I fail to do that I get different errors than in the book.</p></div>

In [44]:
# %run functional_tests.py
!python functional_tests.py

F
FAIL: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 20, in test_can_start_a_list_and_retrieve_it_later
AssertionError: Finish the test!

----------------------------------------------------------------------
Ran 1 test in 6.568s

FAILED (failures=1)


So this is where we remember that we wrote the original functional test to be just a placeholder.  Anyways, that's enough for one chapter; Harry recommends at this point that we commit to the repo, and check out what we've done recently, with:

~~~bash
git diff  # should show our new test in tests.py, and the view in views.py
git commit -am "Basic view now returns minimal HTML"
git log --oneline
~~~